In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.models import Sequential,load_model,Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, SpatialDropout2D
from keras.layers import Input, merge, UpSampling2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/Volumes/solo/ali/pic/train/'
validation_data_dir = '/Volumes/solo/ali/pic/val/'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16
width = 64

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [3]:
model = Sequential()
model.add(Convolution2D(width, (3, 3), padding="same", activation="relu", input_shape=input_shape))
model.add(BatchNormalization(axis = 1))
model.add(Convolution2D(width, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Convolution2D(width*2, (3, 3), padding="same", activation="relu"))
model.add(BatchNormalization(axis = 1))
model.add(Convolution2D(width*2, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(width*4, (3, 3), padding="same", activation="relu"))
model.add(BatchNormalization(axis = 1))
model.add(Convolution2D(width*4, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(width*8, (3, 3), padding="same", activation="relu"))
model.add(BatchNormalization(axis = 1))
model.add(Convolution2D(width*8, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(width*16, (3, 3), padding="same", activation="relu"))
model.add(BatchNormalization(axis = 1))
model.add(Convolution2D(width*16, (3, 3), padding="same", activation="relu"))


model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.35))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

callbacks = [EarlyStopping(monitor='val_loss', patience = 5, verbose = 1),
                 ModelCheckpoint(('/Volumes/solo/ali/Data/model/first_try.h5'), 
                                 monitor='val_loss', 
                                 verbose = 0,
                                 save_best_only = True)]

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks = callbacks)

model.save_weights('/Volumes/solo/ali/Data/model/prob.h5')

Found 19243 images belonging to 2 classes.
Found 531 images belonging to 2 classes.
Epoch 1/50
125/125 [==============================] - 18s - loss: 1.4273 - acc: 0.7510 - val_loss: 7.9259 - val_acc: 0.5083
Epoch 2/50
125/125 [==============================] - 14s - loss: 0.4932 - acc: 0.8130 - val_loss: 0.7051 - val_acc: 0.4897
Epoch 3/50
125/125 [==============================] - 15s - loss: 0.4380 - acc: 0.8330 - val_loss: 0.2084 - val_acc: 0.9720
Epoch 4/50
125/125 [==============================] - 14s - loss: 0.2618 - acc: 0.9220 - val_loss: 0.0648 - val_acc: 0.9695
Epoch 5/50
125/125 [==============================] - 15s - loss: 0.1280 - acc: 0.9630 - val_loss: 2.6171e-04 - val_acc: 1.0000
Epoch 6/50
125/125 [==============================] - 13s - loss: 0.1073 - acc: 0.9775 - val_loss: 0.1642 - val_acc: 0.9574
Epoch 7/50
125/125 [==============================] - 13s - loss: 0.0711 - acc: 0.9820 - val_loss: 0.0272 - val_acc: 0.9961
Epoch 8/50
125/125 [========================

# fenlei 

In [9]:
import time
import gc
import os

import SimpleITK as sitk
import numpy as np
import csv
import scipy
from glob import glob
import pandas as pd
from scipy import ndimage
from tqdm import tqdm 
import pandas as pd

from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature,draw,data
from skimage.segmentation import clear_border
from skimage import data
from scipy import ndimage
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import dicom
import scipy.misc
import numpy as np



import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from numba import autojit
import zarr
from PIL import Image
import cv2
import scipy.spatial.distance as dist
import gc
import warnings
from solo.unet_utils import *
from solo.unet_models import *
#from solo.preproc_utils import *
#from solo.tianchi import *

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from keras import backend as K
K.set_image_dim_ordering('th') 


#from inception_unet import *
np.random.seed(1337)



os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"


warnings.filterwarnings("ignore")

In [10]:
def load_train():
    data_path = src
    folders = [x for x in os.listdir(data_path) if 'subset' in x]
    os.chdir(data_path)
    patients = []
    for i in folders:
        os.chdir(data_path + i)
        #print('Changing folder to: {}'.format(data_path + i))
        patient_ids = [x for x in os.listdir(data_path + i) if '.mhd' in x]
        for id in patient_ids:
            j = '{}/{}'.format(i, id)
            patients.append(j)
    return patients

def get_filename(file_list, case):
    for f in file_list:
        if case in f:
            return(f)
        
def plot_ct_scan(scan):
    f, plots = plt.subplots(int(scan.shape[0] / 20) + 1, 4, figsize=(25, 25))
    for i in range(0, scan.shape[0], 5):
        plots[int(i / 20), int((i % 20) / 5)].axis('off')
        plots[int(i / 20), int((i % 20) / 5)].imshow(scan[i], cmap=plt.cm.bone) 
         
def print_mask(lung_m, nodule_m):
    fig, ax = plt.subplots(1,2, figsize=(20,16))
    ax[0].imshow(lung_m, cmap = plt.cm.bone)
    ax[1].imshow(nodule_m, cmap = plt.cm.bone)
    return

def seq(start, stop, step=1):
    n = int(round((stop - start)/float(step)))
    if n > 1:
        return([start + step*i for i in range(n+1)])
    else:
        return([])
    
def load_itk(filename):
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
    return numpyImage, numpyOrigin, numpySpacing

def world_2_voxel(world_coordinates, origin, spacing):
    stretched_voxel_coordinates = np.absolute(world_coordinates - origin)
    voxel_coordinates = stretched_voxel_coordinates / spacing
    return voxel_coordinates

def voxel_2_world(voxel_coordinates, origin, spacing):
    stretched_voxel_coordinates = voxel_coordinates * spacing
    world_coordinates = stretched_voxel_coordinates + origin
    return world_coordinates

def get_pixels_hu(image):
    image = image.astype(np.int16)
    image[image == threshold_min] = 0
    return np.array(image, dtype=np.int16)

def get_nodule_slices(lung_mask, nodule_mask, lung_raw):
    indexes = np.unique(np.nonzero(nodule_mask)[0])
    #print('Nodule_present on slices: {}'.format(indexes))
    lung_mask_pres = lung_mask[indexes, :, :]
    nod_mask_pres = nodule_mask[indexes, :, :]
    lung_raw_pres = lung_raw[indexes, :, :]    
    return lung_mask_pres, nod_mask_pres, lung_raw_pres

def reshape_3d(image_3d):
    reshaped_img = image_3d.reshape([image_3d.shape[0], 1, 512, 512])
    #print('Reshaped image shape:', reshaped_img.shape)
    return reshaped_img




def create_masks_for_patient_watershed(img_file, save = True):
    def draw_nodule_mask(node_idx, cur_row):
        #print('Working on node: {}, row: {}'.format(node_idx, cur_row), '\n')
        coord_x = cur_row["coordX"]
        coord_y = cur_row["coordY"]
        coord_z = cur_row["coordZ"]
        diam = cur_row["diameter_mm"]
        radius = np.ceil(diam/2)
        noduleRange = seq(-radius, radius, RESIZE_SPACING[0])
        #print('Nodule range:', noduleRange)
        world_center = np.array((coord_z,coord_y,coord_x))   # nodule center
        voxel_center = world_2_voxel(world_center, origin, new_spacing)
        image_mask = np.zeros(lung_img.shape)
        for x in noduleRange:
            for y in noduleRange:
                for z in noduleRange:
                    coords = world_2_voxel(np.array((coord_z+z,coord_y+y,coord_x+x)),origin,new_spacing)
                    if (np.linalg.norm(voxel_center - coords) * RESIZE_SPACING[0]) < radius:
                        image_mask[int(np.round(coords[0])),int(np.round(coords[1])),int(np.round(coords[2]))] = int(1)
        #print(np.max(image_mask))    
      
        return image_mask

    print("Getting mask for image file {}".format(img_file))
    patient_id = img_file.split('/')[-1][:-4]
    mini_df = df_node[df_node["file"] == img_file]
    if mini_df.shape[0] > 0: # some files may not have a nodule--skipping those 
        img, origin, spacing = load_itk(src + img_file)
        height, width = img.shape[1], img.shape[2]
        #calculate resize factor
        RESIZE_SPACING = [1, 1, 1]
        resize_factor = spacing / RESIZE_SPACING
        new_real_shape = img.shape * resize_factor
        new_shape = np.round(new_real_shape)
        real_resize = new_shape / img.shape
        new_spacing = spacing / real_resize
        
        
        lung_img = scipy.ndimage.interpolation.zoom(img, real_resize)
        
        
        
        
        #print('Original image shape: {}'.format(img.shape))
        #print('Resized image shape: {}'.format(lung_img.shape))

        lung_img = get_pixels_hu(lung_img)
        #lung_mask = segment_lung_from_ct_scan(lung_img)
        #lung_mask[lung_mask >= threshold_max] = threshold_max
        #lung_img[lung_img >= threshold_max] = threshold_max
        #lung_img[lung_img == 0] = threshold_min
        
        
        
        lung_mask = lung_img.copy()
        #lung_mask[lung_mask == 0] = threshold_min
        lung_mask[lung_mask >= threshold_max] = threshold_max
        lung_img[lung_img >= threshold_max] = threshold_max
        
        lung_masks_512 = np.zeros([lung_img.shape[0], height_mask, width_mask], dtype = np.float32)
        nodule_masks_512 = np.zeros([lung_img.shape[0], height_mask, width_mask], dtype = np.float32)
        lung_masks_512[lung_masks_512 == 0] = threshold_min
        
        i = 0
        for node_idx, cur_row in mini_df.iterrows(): 
            nodule_mask = draw_nodule_mask(node_idx, cur_row)
            lung_img_512, lung_mask_512, nodule_mask_512 = np.zeros((lung_img.shape[0], 512, 512)), np.zeros((lung_mask.shape[0], 512, 512)), np.zeros((nodule_mask.shape[0], 512, 512))
            lung_mask_512[lung_mask_512 == 0] = threshold_min
            lung_img_512[lung_img_512 == 0] = threshold_min
            original_shape = lung_img.shape
            
            for z in range(lung_img.shape[0]):
                
                offset = (512 - original_shape[1])
                upper_offset = int(np.round(offset/2))
                lower_offset = int(offset - upper_offset)

                new_origin = voxel_2_world([-upper_offset,-lower_offset,0],origin,new_spacing)
                
                lung_img_512[z, upper_offset:-lower_offset,upper_offset:-lower_offset] = lung_img[z,:,:]
                lung_mask_512[z, upper_offset:-lower_offset,upper_offset:-lower_offset] = lung_mask[z,:,:]
                nodule_mask_512[z, upper_offset:-lower_offset,upper_offset:-lower_offset] = nodule_mask[z,:,:]
            nodule_masks_512 += nodule_mask_512

        #print('Offsets shape for node index {} - main: {}, upper: {}, lower: {}'.format(node_idx, offset, upper_offset, lower_offset), '\n')
      
       
        lung_mask_pres, nod_mask_pres, lung_raw_pres = get_nodule_slices(lung_mask_512, nodule_masks_512, lung_img_512)
        #print('Nodules present on slices: ', np.unique(np.nonzero(nodule_masks_512)[0]))
        
        
        
        del lung_mask_512, nodule_masks_512, lung_img_512
        gc.collect()
        
        lung_mask_pres = reshape_3d(lung_mask_pres)
        nod_mask_pres = reshape_3d(nod_mask_pres)
        lung_mask_pres[lung_mask_pres <= threshold_min] = threshold_min
        lung_mask_pres[lung_mask_pres >= threshold_max] = threshold_max
        
        lung_mask_preproc = my_PreProc(lung_mask_pres)
        lung_mask_preproc = lung_mask_preproc.astype(np.float32)
        nod_mask_pres = (nod_mask_pres > 0.0).astype(np.float32)
        nod_mask_pres[nod_mask_pres == 1.0] = 255.

        np.save('{}/lung_mask/{}.npy'.format(dst_nodules, patient_id), lung_mask_preproc)
        np.save('{}/nodule_mask/{}.npy'.format(dst_nodules, patient_id), nod_mask_pres)
        
        del lung_mask_pres, lung_mask_preproc, nod_mask_pres
        gc.collect()
            
        #return lung_mask_preproc, nod_mask_pres
        return
    else: 
        print('\n', 'No nodules found for patient: {}'.format(patient_id), '\n')
        return
    
def predict_on_scan(scan):
    model = unet_model()
    model.load_weights(luna_path + 'Data/model/Unet_fulldatagen.h5')

    num_test = scan.shape[0]
    scan = scan.reshape(num_test, 1, 512, 512)
    imgs_mask_test = np.ndarray([num_test, 1, 512, 512],dtype=np.float32)
    for i in range(num_test):
        imgs_mask_test[i] = model.predict([scan[i:i+1]], verbose=0)[0]
    return imgs_mask_test

In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [12]:
luna_path = '/Volumes/solo/ali/'
npy_path = luna_path + 'Data/test_mask/pre_lung_mask/'
nodemask = luna_path + 'Data/test_mask/pre_nodule_mask/'

pics_path = luna_path + 'pic/test/'

In [13]:
file_list = []
subset_path = os.listdir(npy_path)
for _ in range(len(subset_path)):
    if subset_path[_] != '.DS_Store':
        file_list.append(npy_path + subset_path[_])

In [14]:
df_npy = pd.read_csv('/Volumes/solo/ali/csv/test/annotations_pre.csv',index_col=None)
df_npy["npy_file"] = df_npy["seriesuid"].map(lambda file_name: get_filename(file_list, file_name))

In [20]:
prob_ = []
for node_idx, row in tqdm(df_npy.iterrows()):
    img = np.load(row['npy_file'])
    img = np.squeeze(img)
    x = int(row['coordX'])
    y = int(row['coordY'])
    z = int(row['coordZ'])
    d = int(row['diameter_mm'])
    
    mask = np.zeros([32,32])
    rr, cc=draw.circle(y,x,d)
    draw.set_color(mask,[rr,cc],[1,])
    
    
    
    scipy.misc.imsave(pics_path + str(node_idx) + '.jpg', img[x,y-16:y+16,z-16:z+16]*mask)
    img1 = load_img(pics_path + str(node_idx) + '.jpg')
    x = img_to_array(img1).transpose(1,2,0)
    
    x = x.reshape((1,) + x.shape)
    cc = model.predict(x)
    prob_.append(cc[0][0])

1744it [19:23,  4.34it/s]


In [21]:
bb = pd.read_csv('/Volumes/solo/ali/csv/test/annotations.csv',index_col=None)
final_result = pd.DataFrame({'seriesuid':bb['seriesuid'].values,
             'coordX':bb['coordX'].values,
             'coordY':bb['coordY'].values,
             'coordZ':bb['coordZ'].values,
             'probability':prob_,
                            })
final_result = final_result[['seriesuid','coordX','coordY','coordZ','probability']]
final_result.to_csv('/Volumes/solo/ali/csv/test/annotations_final.csv',index=None)

In [22]:
final_result

,seriesuid,coordX,coordY,coordZ,probability
0,LKDS-00012,-35.927012,-135.243309,-87.900000,3.563873e-29
1,LKDS-00012,-83.693801,-108.198809,-82.633898,3.563873e-29
2,LKDS-00012,-91.620924,-112.620846,-58.400000,3.563873e-29
3,LKDS-00012,-99.054891,-196.064294,50.600000,3.563873e-29
4,LKDS-00012,116.083591,-113.884637,159.375106,3.563873e-29
5,LKDS-00012,95.162698,-211.136371,160.480412,3.563873e-29
6,LKDS-00022,-32.524077,-135.334846,435.500000,3.563873e-29
7,LKDS-00022,42.359000,-161.766000,464.500000,3.563873e-29
8,LKDS-00022,-64.706217,-161.314913,465.500000,3.563873e-29
9,LKDS-00022,40.285829,-66.580024,490.500000,3.563873e-29
